In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils import data
from torchvision import datasets, transforms, models
from torch.optim import lr_scheduler
from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import os

# 이미지 데이터 불러오기

In [2]:
# 구글 드라이브를 마운트 한다.

from google import colab
colab.drive.mount("/drive/")

Drive already mounted at /drive/; to attempt to forcibly remount, call drive.mount("/drive/", force_remount=True).


In [3]:
# 구글 드라이브에서 데이터가 있는 path를 찾았다.

!ls "/drive/My Drive/빅데이터 청년인재 고려대 과정 1조/K-Data 고려대학교 빅데이터 청년인재 교육과정 1조/프로젝트/줄기"

test  train


In [0]:
data_dir = "/drive/My Drive/빅데이터 청년인재 고려대 과정 1조/K-Data 고려대학교 빅데이터 청년인재 교육과정 1조/프로젝트/줄기"

In [5]:
# path 지정함
# os.path.join으로 path 합치는 거 해봄.

data_path = {x:os.path.join(data_dir, x) for x in ["train", "test"]}
data_path

{'test': '/drive/My Drive/빅데이터 청년인재 고려대 과정 1조/K-Data 고려대학교 빅데이터 청년인재 교육과정 1조/프로젝트/줄기/test',
 'train': '/drive/My Drive/빅데이터 청년인재 고려대 과정 1조/K-Data 고려대학교 빅데이터 청년인재 교육과정 1조/프로젝트/줄기/train'}

In [0]:
# Compose는 조립한다는 뜻이다.
# transforms의 함수들을 리스트로 만들어서 인자로 전달하면, pipeline같은 역할을 한다.

# data augmentation도 여기서 처리할 수 있다.
# 하지만 일단 처음 돌리는거이므로 pass.

data_transforms = transforms.Compose([
    transforms.CenterCrop(2688), # 중앙을 중심을 크롭해준다.
                                 # 인자가 int면 정방형, 가로세로 정하려면 시퀀스로.
    transforms.Resize(224),
    transforms.ToTensor(), # tensor 객체로 바꿔준다. 
    transforms.Normalize((0.5, 0.5, 0.5), (0.3, 0.3, 0.3)),# 노말라이즈한다.
                                                           # 평균과 분산을 넣는다. 채널 수 만큼 넣는다.
                                                           # 일단 아무 숫자나 넣어본다.
])

In [7]:
# ImageFolder 함수는 path에서 이미지를 load해온다. 
# ToTensor하기 전에는 PILimage로 가져온다. (이름 정확히 기억 안남)

stem_datasets = {x : torchvision.datasets.ImageFolder(data_path[x], data_transforms) \
              for x in ["train", 'test']}
stem_datasets

{'test': Dataset ImageFolder
     Number of datapoints: 120
     Root location: /drive/My Drive/빅데이터 청년인재 고려대 과정 1조/K-Data 고려대학교 빅데이터 청년인재 교육과정 1조/프로젝트/줄기/test,
 'train': Dataset ImageFolder
     Number of datapoints: 480
     Root location: /drive/My Drive/빅데이터 청년인재 고려대 과정 1조/K-Data 고려대학교 빅데이터 청년인재 교육과정 1조/프로젝트/줄기/train}

In [8]:
# 학습하려면 DataLoader 타입으로 만들어줘야한다.

type(stem_datasets['train'])

torchvision.datasets.folder.ImageFolder

In [9]:
# 폴더 명이 class로 자동 매칭된다.

class_names = stem_datasets['train'].classes
class_names

['감수_train', '대극_train', '파극천_train']

In [10]:
dataset_sizes = {x:len(stem_datasets[x]) for x in ['train', 'test']}
dataset_sizes

{'test': 120, 'train': 480}

## DataLoader 로 넘겨주기

In [11]:
# dataset을 딥러닝 학습할 데이터 셋으로 만들어 준다.... 
# batch_size를 지정할 수 있다.
# shuffle을 True로 하는게 좋다. batch돌면서 랜덤하게 데이터를 준다.
# num_workers가 0이면 모든 코어를 다 쓴다?는거 같음.

# DataLoader한 후에 train, test를 나눌 수 없다. train,test를 split한 다음에 DataLoader로 넘겨줘야한다.

dataloaders = {x: torch.utils.data.DataLoader(stem_datasets[x], batch_size=4, shuffle=True, num_workers=4) \
              for x in ['train', 'test']}
dataloaders

{'test': <torch.utils.data.dataloader.DataLoader at 0x7f230710d128>,
 'train': <torch.utils.data.dataloader.DataLoader at 0x7f230710d2b0>}

In [0]:
# 이건 연습용 코드
# 위와 같은 내용임.

# train = torch.utils.data.DataLoader(stem_images_train, batch_size=4, shuffle=True, num_workers=0)
# test = torch.utils.data.DataLoader(stem_images_test, batch_size=4, shuffle=False, num_workers=0)

In [13]:
# DataLoader 하기 전과 type이 바뀜.

type(dataloaders['train'])

torch.utils.data.dataloader.DataLoader

# vgg pre-trained 가져오기
- 가져왔지만 안 씀

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [15]:
vgg_model = torchvision.models.vgg16_bn(pretrained=True).to(device) # 기존에 만들어진 vgg network 을 이미지넷 데이터에 트레이닝해둔 파라미터를 그대로 받아옵니다.

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 553507836/553507836 [00:05<00:00, 96341487.46it/s] 


In [16]:
vgg_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

In [17]:
# vgg_model.features 는 vgg모델에서 feature extraction을 담당하는 부분만 sequential된 부분이다.
# children()하면 generator로 바꿔준다.

[x for x in vgg_model.features.children()]

[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace),
 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace),
 Conv2d(2

# 학습모델

In [0]:
def train_network(net,optimizer,trainloader, epochs=5):
    for epoch in range(epochs):  # loop over the dataset multiple times
                                 # epochs = 전체 데이터가 한바퀴 돈 것
        running_loss = 0.0 # running loss를 저장하기 위한 변수입니다. 
        for i, data in enumerate(trainloader): # 한 Epoch 만큼 돕니다. 매 iteration 마다 정해진 Batch size 만큼 데이터를 뱉습니다. 
            # get the inputs
            inputs, labels = data # DataLoader iterator의 반환 값은 input_data 와 labels의 튜플 형식입니다. 
            inputs = inputs.to(device) # Pytorch에서 nn.Module 에 넣어 Backprop을 계산 하기 위해서는 gpu 연동을 이와 같이 해줘야 합니다.
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()    #  현재 기존의 backprop을 계산하기 위해서 저장했던 activation buffer 를 비웁니다.
            # forward + backward + optimize
            outputs = net(inputs) # input 을 넣은 위 network 로 부터 output 을 얻어냅니다. 
            loss = criterion(outputs, labels) # loss fucntion에 주어진 target과 output 의 score를 계산하여 반환합니다. 
            loss.backward() # * Scalar Loss value를 Backward() 해주게 되면 주어진 loss값을 바탕으로 backpropagation이 진행됩니다. 
            optimizer.step() # 계산된 Backprop 을 바탕으로 optimizer가 gradient descenting 을 수행합니다. 

            # print statistics
            running_loss += loss.item()
            if (i+1) % 10 == 0:    # print every 500 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

    print('Finished Training')

In [0]:
def test(model,test_loader):
    model.eval() # Eval Mode 왜 해야 할까요?  --> nn.Dropout BatchNorm 등의 Regularization 들이 test 모드로 들어가게 되기 때문입니다. 
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)  # 기존의 train function의 data 처리부분과 같습니다. 
        output = model(data) 
        pred = output.max(1, keepdim=True)[1] # get the index of the max 
        correct += pred.eq(target.view_as(pred)).sum().item() # 정답 데이터의 갯수를 반환합니다. 

    test_loss /= len(test_loader.dataset)
    print('\nTest set:  Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
class vgg16_my(nn.Module):
    def __init__(self):
        super(vgg16_my, self).__init__()
        self.feature_extraction = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(64,64,3,1,1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64,128,3,1,1),
            nn.Conv2d(128,128,3,1,1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128,256,3,1,1),
            nn.Conv2d(256,256,3,1,1),
            nn.Conv2d(256,256,3,1,1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256,512,3,1,1),
            nn.Conv2d(512,512,3,1,1),
            nn.Conv2d(512,512,3,1,1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512,512,3,1,1),
            nn.Conv2d(512,512,3,1,1),
            nn.Conv2d(512,512,3,1,1),
            nn.MaxPool2d(2,2),
        )
          
        # fc는 내가 필요한대로 새로 만들어야 한다.
        self.fc = nn.Sequential(  # 기존에는 이미지넷에 학습되어있기 때문에, 이를 내 데이터셋용으로 바꿔줄 필요가 있습니다. 
            nn.Linear(25088, 4096),   # 따라서 1000이 아닌 10가지의 클래스만을 대상으로 하는 linear 레이어를 새로 쌓고 학습시켜주는부분입니다.
            nn.ReLU(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, 3),
        )
        
    def forward(self, x):
        out = self.feature_extraction(x) # 512,7,7
        out = out.view(x.size(0), -1)
        out = self.fc(out)
#         out = nn.Softmax(out)
        return out

In [21]:
vgg16_my()

vgg16_my(
  (feature_extraction): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): Con

In [0]:
lr = 0.001
device = 'cuda' if torch.cuda.is_available() else "cpu"
vgg16_my = vgg16_my().to(device)
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(vgg16_my.parameters(), lr=lr) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [24]:
train_network(vgg16_my,optimizer,dataloaders["train"], epochs=1)

[1,   100] loss: 84773228.627
Finished Training


In [25]:
test(vgg16_my,dataloaders["test"])


Test set:  Accuracy: 40/120 (33%)



In [0]:
import pickle

In [28]:
with open('/drive/My Drive/Colab Notebooks/pure_vgg16.pickle', 'wb') as file:    # 확장자 마음대로
    pickle.dump(vgg16_my, file)

PicklingError: ignored

In [0]:
torch.save(vgg16_my.state_dict(), "/drive/My Drive/Colab Notebooks/pure_vgg16.pth")